In [1]:
X_DIM = 256
Y_DIM = 256
UNPROCESSED_DIRECTORY = "CNN_synth_testset"
PROCESSED_DIRECTORY = "processed_images"
CROP = False
import os
import numpy as np
from PIL import Image as im_lib
from PIL import ImageFilter
from keras.layers import Layer
from keras import models
from keras import layers
from keras.utils import Sequence
from keras.metrics import Metric
import keras.backend as backend
import tensorflow as tf
from math import ceil
from sklearn.model_selection import train_test_split
import time

In [2]:
def obtain_image_paths(unprocessed_directory = UNPROCESSED_DIRECTORY):
    data_directory = os.listdir("./"+unprocessed_directory)
    list_of_image_paths = []
    for dirpath, dirname, filename in os.walk("./"+unprocessed_directory):
        if not dirname:
            for dirpath2, dirname2, filename2 in os.walk(dirpath):
                for filedir in filename2:
                    list_of_image_paths.append(dirpath+"/"+filedir)
    return list_of_image_paths

In [3]:
#Create constants and import libraries
def categorical_output(image_path):
    keywords = ["biggan", "crn", "cyclegan","deepfake","gaugan","imle","progan","san","seeingdark","stargan","stylegan2", "stylegan",
           "whichfaceisreal"]
    category = [0] * (len(keywords) + 1)
    for kwd_index in range(len(keywords)):
        kwd = keywords[kwd_index]
        if kwd in image_path:
            category[kwd_index] = 1
            return category
    category[-1] = 1
    return category
class ImageBatchSequence(Sequence):
    def __init__(self, x_set, batch_size, x_dim = X_DIM, y_dim = Y_DIM, filter_function = None, 
                 output_function = None, normalizing_factor = 255):
        self.x = x_set
        if output_function is None:
            y_set = [(1 if "fake" in image_path else 0) for image_path in x_set] #This makes the output to be 1 (AI-generated) if FAKE is in pathname.
            #                                                  # Otherwise, it makes the output 0 (not AI-generated)
        else:
            y_set = [output_function(image_path) for image_path in x_set]
        self.y = y_set
        self.batch_size = batch_size
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.filter_function = filter_function
        self.normalizing_factor = normalizing_factor
    def __len__(self):
        return ceil(len(self.x)/self.batch_size)
    def __getitem__(self, index):
        low = index * self.batch_size
        high = min(low+self.batch_size, len(self.x))
        batch_x = self.x[low : high]
        batch_y = self.y[low : high]
        image_x_batch = []
        for image_path in batch_x:
            image = im_lib.open(image_path)
            if self.filter_function is not None:
                image = self.filter_function(image)
            image_array = np.asarray(image).reshape(self.x_dim,self.y_dim,3)/self.normalizing_factor
            image_x_batch.append(image_array)
        return np.array(image_x_batch), np.array(batch_y)

In [4]:
X = obtain_image_paths("Test1_modular")
y = [categorical_output(x) for x in X]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                       shuffle = True,
                                                       random_state = 440,
                                                       test_size = .2, stratify = y)

In [6]:
keywords = ["biggan", "crn", "cyclegan","deepfake","gaugan","imle","progan","san","seeingdark","stargan", "stylegan",
           "stylegan2","whichfaceisreal"]
for kwd in keywords:
    print(kwd+":"+str(y_train.count(categorical_output(kwd))))
print("real : "+str(y_train.count(categorical_output("fuck"))))
print(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;")
for kwd in keywords:
    print(kwd+":"+str(y_test.count(categorical_output(kwd))))
print("real : "+str(y_test.count(categorical_output("fuck"))))

biggan:1600
crn:5106
cyclegan:1057
deepfake:4324
gaugan:4000
imle:5105
progan:3200
san:174
seeingdark:144
stargan:1599
stylegan:4793
stylegan2:6390
whichfaceisreal:800
real : 33960
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
biggan:400
crn:1276
cyclegan:264
deepfake:1081
gaugan:1000
imle:1277
progan:800
san:43
seeingdark:36
stargan:400
stylegan:1198
stylegan2:1598
whichfaceisreal:200
real : 8490


In [7]:
X_train_train,X_val,y_train_train,y_val = train_test_split(X_train, y_train,
                                                          test_size=.2,
                                                          shuffle=True,
                                                          stratify=y_train,
                                                          random_state=440)

In [8]:
model = models.Sequential()
model.add(layers.Conv2D( 32, (3,3), activation='relu', input_shape=(256,256,3,)))
model.add( layers.MaxPooling2D( (2,2), strides = 2 ) )
model.add( layers.Conv2D(64, (3,3), activation='relu'))
model.add( layers.MaxPooling2D( (2,2), strides=2) )
model.add( layers.Flatten() )
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(14, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 64)                1

In [9]:
class MulticlassPrecisionMetric(Metric):
    def __init__(self, name = "mp", category_tested = 0):
        super(MulticlassPrecisionMetric,self).__init__(name = name)
        self.category_tested = category_tested
        self.multi_true_positives = self.add_weight(name = 'tp' + str(self.category_tested), initializer = 'zeros')
        self.multi_tf_positives = self.add_weight(name = 'tfp' + str(self.category_tested), initializer = 'zeros')
        self.mp = self.add_weight(name = 'mp' + str(self.category_tested), initializer = 'zeros')
    def update_state(self, y_true, y_pred, sample_weight = None):
        y_true = tf.argmax(y_true, axis = -1)
        y_pred = tf.argmax(y_pred, axis = -1)
        tp = tf.reduce_sum(\
                         tf.cast(tf.equal(y_true,self.category_tested),tf.float32)\
                         *tf.cast(tf.equal(y_pred,self.category_tested),tf.float32))
        self.multi_true_positives.assign_add(tp)
        self.multi_tf_positives.assign_add(\
                                           tf.reduce_sum(\
                                                       tf.cast(tf.equal(y_pred,self.category_tested),tf.float32)))
        self.mp.assign(self.multi_true_positives/(self.multi_tf_positives+backend.epsilon()))
    def result(self):
        return self.mp

In [10]:
class MulticlassRecallMetric(Metric):
    def __init__(self, name = "mp", category_tested = 0):
        super(MulticlassRecallMetric,self).__init__(name = name)
        self.category_tested = category_tested
        self.multi_true_positives = self.add_weight(name = 'tp' + str(self.category_tested), initializer = 'zeros')
        self.multi_predic = self.add_weight(name = 'tpr' + str(self.category_tested), initializer = 'zeros')
        self.mr = self.add_weight(name = 'mr' + str(self.category_tested), initializer = 'zeros')
    def update_state(self, y_true, y_pred,sample_weight = None):
        y_true = tf.argmax(y_true, axis = -1)
        y_pred = tf.argmax(y_pred, axis = -1)
        tp = tf.reduce_sum(\
                         tf.cast(tf.equal(y_true,self.category_tested),tf.float32)\
                         *tf.cast(tf.equal(y_pred,self.category_tested),tf.float32))
        self.multi_true_positives.assign_add(tp)
        self.multi_predic.assign_add(\
                                           tf.reduce_sum(\
                                                       tf.cast(tf.equal(y_true,self.category_tested),tf.float32)))
        self.mr.assign(self.multi_true_positives/(self.multi_predic+backend.epsilon()))
    def result(self):
        return self.mr

In [11]:
keywords = ["biggan", "crn", "cyclegan","deepfake","gaugan","imle","progan","san","seeingdark","stargan","stylegan2",
            "stylegan", "whichfaceisreal"]
METRICS = []
for kwd_index in range(len(keywords)):
    precision_metric = MulticlassPrecisionMetric(name = "precision_"+keywords[kwd_index], 
                                                 category_tested = kwd_index)
    recall_metric = MulticlassRecallMetric(name = "recall_"+keywords[kwd_index], 
                                                 category_tested = kwd_index)
    METRICS.append(precision_metric)
    METRICS.append(recall_metric)
precision_metric = MulticlassPrecisionMetric(name = "precision_real", category_tested = len(keywords))
recall_metric = MulticlassRecallMetric(name = "recall_real", category_tested = len(keywords))
METRICS.append(precision_metric)
METRICS.append(recall_metric)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = [METRICS])

In [12]:
def HPF_filter(image):
    return im_lib.fromarray(np.asarray(image)-np.asarray(image.filter(ImageFilter.GaussianBlur)))

In [13]:
train_sequence = ImageBatchSequence(X_train_train, batch_size = 32, filter_function = HPF_filter, 
                                    output_function = categorical_output)
validation_sequence = ImageBatchSequence(X_val, batch_size = 32, filter_function = HPF_filter, 
                                    output_function = categorical_output)

In [ ]:
model.fit(train_sequence, epochs = 10, validation_data = validation_sequence)

Epoch 1/10
1807/1807 [==============================] - 421s 229ms/step - loss: 0.8426 - precision_biggan: 0.2388 - recall_biggan: 0.0250 - precision_crn: 0.9753 - recall_crn: 0.8906 - precision_cyclegan: 0.3772 - recall_cyclegan: 0.0508 - precision_deepfake: 0.9100 - recall_deepfake: 0.9295 - precision_gaugan: 0.5054 - recall_gaugan: 0.3097 - precision_imle: 0.9312 - recall_imle: 0.8847 - precision_progan: 0.7083 - recall_progan: 0.2570 - precision_san: 0.1667 - recall_san: 0.0360 - precision_seeingdark: 0.7143 - recall_seeingdark: 0.3043 - precision_stargan: 0.8810 - recall_stargan: 0.8163 - precision_stylegan2: 0.6076 - recall_stylegan2: 0.5016 - precision_stylegan: 0.7317 - recall_stylegan: 0.5263 - precision_whichfaceisreal: 0.4951 - recall_whichfaceisreal: 0.0797 - precision_real: 0.7154 - recall_real: 0.9262 - val_loss: 0.4400 - val_precision_biggan: 0.5333 - val_recall_biggan: 0.1250 - val_precision_crn: 0.9961 - val_recall_crn: 0.9980 - val_precision_cyclegan: 0.6408 - val_rec

Epoch 6/10
1807/1807 [==============================] - 398s 220ms/step - loss: 0.0208 - precision_biggan: 0.9914 - recall_biggan: 0.9906 - precision_crn: 0.9983 - recall_crn: 0.9985 - precision_cyclegan: 0.9833 - recall_cyclegan: 0.9752 - precision_deepfake: 0.9942 - recall_deepfake: 0.9951 - precision_gaugan: 0.9966 - recall_gaugan: 0.9953 - precision_imle: 0.9988 - recall_imle: 0.9985 - precision_progan: 0.9860 - recall_progan: 0.9875 - precision_san: 0.9420 - recall_san: 0.9353 - precision_seeingdark: 0.9741 - recall_seeingdark: 0.9826 - precision_stargan: 0.9976 - recall_stargan: 0.9953 - precision_stylegan2: 0.9908 - recall_stylegan2: 0.9904 - precision_stylegan: 0.9927 - recall_stylegan: 0.9893 - precision_whichfaceisreal: 0.9874 - recall_whichfaceisreal: 0.9766 - precision_real: 0.9947 - recall_real: 0.9958 - val_loss: 0.7249 - val_precision_biggan: 0.5046 - val_recall_biggan: 0.3406 - val_precision_crn: 0.9990 - val_recall_crn: 0.9990 - val_precision_cyclegan: 0.8137 - val_rec